In [1]:

import matplotlib
from matplotlib import pyplot
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandasql import sqldf
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
# be able to save images on server
matplotlib.use('Agg')


/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9151/1460567149.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


## Air quality anno 2014-2020

In [2]:
airQuality = read_csv("AirQuality1420.csv")
airQuality.drop(["CO", "NO2", "NOx", "Ozone",
                "Sulfur_dioxide"], axis=1, inplace=True)


airQuality


,IDStation,NameStation,Date,Ammonia,PM10,PM25
0,677,Cremona Via Fatebenefratelli,2014-01-01,2.5,73.0,66.0
1,677,Cremona Via Fatebenefratelli,2014-01-02,4.5,51.0,NaN
2,677,Cremona Via Fatebenefratelli,2014-01-03,1.6,47.0,46.0
3,677,Cremona Via Fatebenefratelli,2014-01-04,1.1,42.0,38.0
4,677,Cremona Via Fatebenefratelli,2014-01-05,0.7,21.0,21.0
...,...,...,...,...,...,...
2543,677,Cremona Via Fatebenefratelli,2020-12-27,7.3,35.0,31.0
2544,677,Cremona Via Fatebenefratelli,2020-12-28,6.7,NaN,20.0
2545,677,Cremona Via Fatebenefratelli,2020-12-29,6.9,46.0,45.0
2546,677,Cremona Via Fatebenefratelli,2020-12-30,7.3,62.0,60.0


## Weather 2014-2015

In [3]:
w14_15 = read_csv("Weather14-15.csv")
w16_17 = read_csv("Weather16-17.csv")
w18_19 = read_csv("Weather18-19.csv")
w20    = read_csv("Weather20.csv")

### Unione delle tabelle del meteo

In [4]:
weather = sqldf('''
                SELECT *
                FROM w14_15
                UNION
                 SELECT *
                FROM w16_17
                UNION
                SELECT *
                FROM w18_19
                UNION
                SELECT *
                FROM w20  
                ''')
a = 730+726+729+366
print("Check per vedere se le dimensioni tornano: "+str(a))
weather


Check per vedere se le dimensioni tornano: 2551


,IDStation,NameStation,Date,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,677,Cremona Via Fatebenefratelli,2014-01-01,NaN,NaN,2.233333,88.487500,56.600000,0.0
1,677,Cremona Via Fatebenefratelli,2014-01-02,NaN,NaN,3.658333,96.533333,9.841667,0.6
2,677,Cremona Via Fatebenefratelli,2014-01-03,NaN,NaN,4.029167,99.662500,12.537500,1.0
3,677,Cremona Via Fatebenefratelli,2014-01-04,NaN,NaN,4.691667,99.900000,4.316667,16.6
4,677,Cremona Via Fatebenefratelli,2014-01-05,NaN,NaN,5.412500,99.900000,7.887500,19.2
...,...,...,...,...,...,...,...,...,...
2546,677,Cremona Via Fatebenefratelli,2020-12-27,0.045293,4.0,0.952083,92.725000,58.411806,0.0
2547,677,Cremona Via Fatebenefratelli,2020-12-28,0.212185,321.0,0.277465,99.597183,8.239437,0.0
2548,677,Cremona Via Fatebenefratelli,2020-12-29,0.451607,275.0,-0.663889,99.598611,28.943750,0.0
2549,677,Cremona Via Fatebenefratelli,2020-12-30,0.634008,253.0,1.771528,99.568056,29.232639,0.0


# Join per creare la tabella che ci interessa

In [5]:
final = sqldf('''SELECT *
         FROM airQuality AS a JOIN weather AS w
         ON a.Date =  w.Date
         ''')

print("Dimensioni qualità aria: "+str(airQuality.shape))
print("Dimensioni meteo: "+str(weather.shape))
print("Dimensioni matrice finale"+str(final.shape))
final


Dimensioni qualità aria: (2548, 6)
Dimensioni meteo: (2551, 9)
Dimensioni matrice finale(2548, 15)


,IDStation,NameStation,Date,Ammonia,PM10,PM25,IDStation,NameStation,Date,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,677,Cremona Via Fatebenefratelli,2014-01-01,2.5,73.0,66.0,677,Cremona Via Fatebenefratelli,2014-01-01,NaN,NaN,2.233333,88.487500,56.600000,0.0
1,677,Cremona Via Fatebenefratelli,2014-01-02,4.5,51.0,NaN,677,Cremona Via Fatebenefratelli,2014-01-02,NaN,NaN,3.658333,96.533333,9.841667,0.6
2,677,Cremona Via Fatebenefratelli,2014-01-03,1.6,47.0,46.0,677,Cremona Via Fatebenefratelli,2014-01-03,NaN,NaN,4.029167,99.662500,12.537500,1.0
3,677,Cremona Via Fatebenefratelli,2014-01-04,1.1,42.0,38.0,677,Cremona Via Fatebenefratelli,2014-01-04,NaN,NaN,4.691667,99.900000,4.316667,16.6
4,677,Cremona Via Fatebenefratelli,2014-01-05,0.7,21.0,21.0,677,Cremona Via Fatebenefratelli,2014-01-05,NaN,NaN,5.412500,99.900000,7.887500,19.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,677,Cremona Via Fatebenefratelli,2020-12-27,7.3,35.0,31.0,677,Cremona Via Fatebenefratelli,2020-12-27,0.045293,4.0,0.952083,92.725000,58.411806,0.0
2544,677,Cremona Via Fatebenefratelli,2020-12-28,6.7,NaN,20.0,677,Cremona Via Fatebenefratelli,2020-12-28,0.212185,321.0,0.277465,99.597183,8.239437,0.0
2545,677,Cremona Via Fatebenefratelli,2020-12-29,6.9,46.0,45.0,677,Cremona Via Fatebenefratelli,2020-12-29,0.451607,275.0,-0.663889,99.598611,28.943750,0.0
2546,677,Cremona Via Fatebenefratelli,2020-12-30,7.3,62.0,60.0,677,Cremona Via Fatebenefratelli,2020-12-30,0.634008,253.0,1.771528,99.568056,29.232639,0.0


# Export del file 

In [6]:
import os

In [7]:
os.chdir('/Users/marcovinciguerra/Github/GitTesi/Python/Cremona')


In [8]:
final.to_csv('RisultatiCremona.csv')


In [9]:
os.chdir('/Users/marcovinciguerra/Github/GitTesi/Python/Cremona/QuadrantiCremona')
final.to_csv('RisultatiCremona.csv')
